In [2]:
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

In [132]:
# CO2 emissions per combustion engine car per month
SINGLE_CAR_EMISSIONS = float(4.6/12)

# CA hourly electricity demand in MWh, 2015-2022
ca_hourly_demand = pd.read_csv('data/original/ca_hourly_demand.csv')

# CA monthly electricity demand in thousand of MWh, 2001-2022
ca_monthly_supply = pd.read_csv('data/original/ca_monthly_supply.csv')

# CA monthly CO2 emissions in MTCO2, 2014-2022
ca_monthly_emissions = pd.read_csv('data/original/ca_monthly_emissions.csv')

# CA total yearly vehicle registrations, 2016-2021
ca_yearly_vehicles = pd.read_csv('data/original/ca_yearly_vehicles.csv')

# Weather data
all_states_precip = pd.read_csv('data/original/all_states_monthly_precip.csv')
all_states_tmax = pd.read_csv('data/original/all_states_monthly_max_temp.csv')
all_states_tmin = pd.read_csv('data/original/all_states_monthly_min_temp.csv')

fuel_types = ['electric','plug-in hybrid electric','fuel cell','diesel','flex fuel','gasoline','gasoline hybrid','natural gas','propane']

months_list = ['January','February','March','April','May','June','July','August','September','October','November','December']

FileNotFoundError: [Errno 2] No such file or directory: 'data/original/ca_yearly_vehicles.csv'

In [35]:
def cleanup_null_lines(df: pd.DataFrame, col):
    df = df.dropna()
    df = df[df[col] != '']
    df = df[df[col] > 0]
    return df

**ca_monthly_supply.csv**

remove unnecessary columns

In [192]:
ca_monthly_supply_clean = ca_monthly_supply
eliminated = ['date1','date2','date3','date4','date5']
ca_monthly_supply_clean = ca_monthly_supply.drop(columns=eliminated)

ca_monthly_supply_clean.to_csv('data/original/ca_monthly_supply.csv')
ca_monthly_supply_clean.head()

,date,coal,natural gas,nuclear,hydroelectric,wind,solar
0,202209,24.98724,10736.90153,1611.663,1732.27996,887.64266,3465.29041
1,202208,19.65788,11568.06973,1679.400,1756.68800,1083.22678,3985.04144
2,202207,22.38795,9504.45244,1684.102,1856.69645,1443.68919,4286.62626
3,202206,19.56353,7134.78680,1633.288,1816.49729,1460.87295,4455.16726
4,202205,15.52536,5208.03935,1692.998,1835.48466,1814.01024,4217.61698


**ca_monthly_weather.csv**

In [16]:
cols = ['year','county code'] + months_list
ca_monthly_precip = pd.DataFrame(columns=cols)

for index, row in all_states_precip.iterrows():
    code = str(int(row['code']))
    if len(code) < 11:
        code = '0' + code
    state_code = code[0:2]
    curr_year = int(code[7:])
    if state_code == '04' and curr_year > 2014:
        curr_county = code[2:5]
        new_data = []
        new_data.append(curr_year)
        new_data.append(curr_county)
        for month in months_list:
            new_data.append(row[month])
        new_frame = pd.DataFrame([new_data], columns=cols)
        ca_monthly_precip = ca_monthly_precip.append(new_frame)

ca_monthly_precip.to_csv('data/cleaned/monthly_precip.csv')
print(ca_monthly_precip.shape)
ca_monthly_precip.head()

(464, 14)


,year,county code,January,February,March,April,May,June,July,August,September,October,November,December
0,2015,001,0.01,2.25,0.26,1.15,0.37,0.27,0.01,0.02,0.04,0.08,2.41,3.52
0,2016,001,5.65,0.84,5.25,1.94,0.24,0.00,0.00,0.00,0.00,3.65,1.48,3.50
0,2017,001,9.38,7.41,2.65,2.50,0.03,0.04,0.00,0.01,0.06,0.21,2.58,0.10
0,2018,001,4.28,0.57,4.39,2.44,0.02,0.00,0.00,0.00,0.00,0.14,2.71,2.02
0,2019,001,3.73,6.83,4.11,0.47,2.25,0.00,0.00,0.00,0.19,0.00,1.09,3.62


In [64]:
cols = ['year','county code'] + months_list
ca_monthly_tmax = pd.DataFrame(columns=cols)

for index, row in all_states_tmax.iterrows():
    code = str(int(row['code']))
    if len(code) < 11:
        code = '0' + code
    state_code = code[0:2]
    curr_year = int(code[7:])
    if state_code == '04' and curr_year > 2014:
        curr_county = code[2:5]
        new_data = []
        new_data.append(curr_year)
        new_data.append(curr_county)
        for month in months_list:
            new_data.append(row[month])
        new_frame = pd.DataFrame([new_data], columns=cols)
        ca_monthly_tmax = ca_monthly_tmax.append(new_frame)

ca_monthly_tmax.to_csv('data/cleaned/monthly_tmax.csv')
print(ca_monthly_tmax.shape)
ca_monthly_tmax.head()

(464, 14)


,year,county code,January,February,March,April,May,June,July,August,September,October,November,December
0,2015,001,61.0,65.4,70.8,69.60,68.80,81.90,82.80,83.7,84.0,78.7,60.8,54.8
0,2016,001,56.2,65.5,64.1,69.60,73.60,82.80,83.80,81.9,81.6,71.6,64.1,55.4
0,2017,001,53.6,58.2,65.4,66.80,74.90,81.20,86.80,84.0,83.0,76.6,63.6,59.9
0,2018,001,58.7,62.5,61.3,67.30,70.60,80.80,85.90,82.2,80.4,75.5,66.2,56.6
0,2019,001,58.4,53.3,60.6,69.80,68.40,81.80,83.10,85.6,80.6,75.1,67.0,56.4


In [73]:
CA_CODE = '04'
cols = ['year','county code'] + months_list
ca_monthly_tmin = pd.DataFrame(columns=cols)

for index, row in all_states_tmin.iterrows():
    code = str(int(row['code']))
    if len(code) < 11:
        code = '0' + code
    state_code = code[0:2]
    curr_year = int(code[7:])
    if state_code == CA_CODE and curr_year > 2014:
        curr_county = code[2:5]
        new_data = []
        new_data.append(curr_year)
        new_data.append(curr_county)
        for month in months_list:
            new_data.append(row[month])
        new_frame = pd.DataFrame([new_data], columns=cols)
        ca_monthly_tmin = ca_monthly_tmin.append(new_frame)

ca_monthly_tmin.to_csv('data/cleaned/monthly_tmin.csv')
print(ca_monthly_tmin.shape)
ca_monthly_tmin.head()

(464, 14)


,year,county code,January,February,March,April,May,June,July,August,September,October,November,December
0,2015,001,41.5,46.0,47.4,46.4,49.5,56.8,60.5,59.7,58.3,55.9,41.6,39.2
0,2016,001,43.8,44.8,46.1,48.6,51.9,55.9,57.2,57.3,55.3,51.8,46.5,39.1
0,2017,001,39.4,43.9,45.6,46.7,50.9,55.8,59.2,60.2,58.9,51.1,46.7,39.5
0,2018,001,43.6,40.5,42.6,46.2,49.9,54.0,59.4,57.4,54.3,51.8,45.7,41.8
0,2019,001,43.2,39.0,43.1,48.8,49.1,56.2,57.8,60.0,56.8,49.5,45.0,43.4


**hourly_emissions_demand.csv**

Compile file (emissions_demand.csv) with CA HOURLY CO2 emissions and hourly electricity demand

In [48]:
DATE_START_IDX = 10
DATE_END_IDX = 18

emissions_demand = ca_hourly_demand
emissions_demand['emissions'] = ''

path_name = 'data/ca_daily_emissions'
cwd = os.path.abspath(path_name)
file_list = os.listdir(cwd)

# Sum hourly emissions values from 5-minute interval files
for file in file_list:
    curr_daily_file = pd.read_csv(path_name + '/' + file)
    curr_date = file[DATE_START_IDX:DATE_END_IDX]
    for i in range(24):
        if i > 0: # Does not include 0h values for simplicity
            total_hour_demand = 0
            curr_hour = str(i)
            prev_hour = str(i - 1)
            for i in range(12):
                five_min_increment = str(i * 5)
                if len(five_min_increment) < 2:
                    if i == 0:
                        full_string_hour = curr_hour + ':00'
                    else:
                        full_string_hour = prev_hour + ':0' + five_min_increment
                else:
                    full_string_hour = prev_hour + ':' + five_min_increment
                if full_string_hour not in list(curr_daily_file.columns):
                    full_string_hour = '0' + full_string_hour
                total_hour_demand += curr_daily_file[full_string_hour][0]
            if len(curr_hour) < 2:
                curr_hour = '0' + curr_hour
            full_date = curr_date + 'T' + curr_hour + 'Z'
            emissions_demand.loc[emissions_demand['date'] == full_date, 'emissions'] = total_hour_demand

emissions_demand = cleanup_null_lines(emissions_demand, 'emissions')

emissions_demand.to_csv('data/hourly_emissions_demand.csv')
emissions_demand.head()

,date,demand,emissions
0,20221127T23Z,23637,84405
1,20221127T22Z,23599,87295
2,20221127T21Z,23323,88500
3,20221127T20Z,23718,88878
4,20221127T19Z,25038,90162


**yearly_all_data.csv**

Compile file (emissions_vehicles.csv) with CA YEARLY CO2 emissions and number of registered vehicles

In [23]:
emissions_vehicles = ca_yearly_vehicles
emissions_vehicles['emissions'] = ''

months_list = list(ca_monthly_emissions)
months_list.remove('year')

year_list = list(ca_monthly_emissions['year'])

for year in year_list: 
    annual_emissions = ca_monthly_emissions.loc[ca_monthly_emissions['year'] == year][months_list].sum(axis=1)
    emissions_vehicles.loc[emissions_vehicles['year'] == year, 'emissions'] = float(annual_emissions)

emissions_vehicles = cleanup_null_lines(emissions_vehicles, 'emissions')
emissions_vehicles.to_csv('data/emissions_vehicles.csv')

emissions_vehicles


,year,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane,emissions
0,2021,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,53203603.23
1,2020,369364,259109,7129,700366,1144536,25021380,1155477,7676,897,49626238.88
2,2019,308447,251522,6933,680622,1131474,25526368,1114944,8462,1015,51049895.52
3,2018,232239,216328,5525,657857,1122312,25377093,1060166,8910,1063,53938026.76
4,2017,159705,163057,3271,640281,1108886,25313468,1019085,9212,1074,52856896.33
5,2016,115569,114602,854,588872,948746,22661954,860599,8521,966,57959115.485
6,2015,82686,83482,79,555652,878165,22026350,776480,8500,895,66241778.651
7,2014,50175,58764,38,524118,817338,21737732,701945,8448,946,68780731.98


Add electricity demand to above file (yearly_all_data.csv)

In [511]:
ev = pd.read_csv('data/emissions_vehicles.csv')

yearly_all_data = ev
yearly_all_data['demand'] = ''

yearly_demand = {}
for index, row in ca_hourly_demand.iterrows():
    curr_year = row['date'][0:4]
    if curr_year in yearly_demand:
        yearly_demand[curr_year] += row['demand']
    else:
        yearly_demand[curr_year] = row['demand']

for year in yearly_demand:
    yearly_all_data.loc[yearly_all_data['year'] == int(year), 'demand'] = yearly_demand[year]

yearly_all_data = cleanup_null_lines(yearly_all_data, 'demand')
yearly_all_data.to_csv('data/yearly_all_data.csv')
yearly_all_data.head()


,Unnamed: 0,year,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane,emissions,demand
0,0,2021,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,53203603.23,269006652
1,1,2020,369364,259109,7129,700366,1144536,25021380,1155477,7676,897,49626238.88,268529517
2,2,2019,308447,251522,6933,680622,1131474,25526368,1114944,8462,1015,51049895.52,264368288
3,3,2018,232239,216328,5525,657857,1122312,25377093,1060166,8910,1063,53938026.76,275818547
4,4,2017,159705,163057,3271,640281,1108886,25313468,1019085,9212,1074,52856896.33,284621714


**yearly_vehicles_total.csv**

In [63]:
yearly_vehicles_total = ca_yearly_vehicles
yearly_vehicles_total['total'] = ''

vehicle_list = list(yearly_vehicles_total)
vehicle_list.remove('year')

for index, row in yearly_vehicles_total.iterrows():
    year = row['year']
    annual_vehicles = yearly_vehicles_total.loc[yearly_vehicles_total['year'] == year][vehicle_list].sum(axis=1)
    yearly_vehicles_total.loc[yearly_vehicles_total['year'] == year, 'total']= int(annual_vehicles)

yearly_vehicles_total.to_csv('data/yearly_vehicles_total.csv')
yearly_vehicles_total.head()

,year,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane,total
0,2021,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,29942517
1,2020,369364,259109,7129,700366,1144536,25021380,1155477,7676,897,28665934
2,2019,308447,251522,6933,680622,1131474,25526368,1114944,8462,1015,29029787
3,2018,232239,216328,5525,657857,1122312,25377093,1060166,8910,1063,28681493
4,2017,159705,163057,3271,640281,1108886,25313468,1019085,9212,1074,28418039


**monthly_all_data.csv**

Compile file (monthly_all_data.csv) with CA MONTHLY electricity demand, car registrations, and CO2 emissions

In [67]:
# Sum total electricity generation for each month, converted to megawatthours

CONVERSION_FACTOR = 1000

monthly_all_data = ca_monthly_supply
col_list = ['coal','natural gas','nuclear','hydroelectric','wind']
monthly_all_data['total'] = ''
monthly_all_data['total'] = monthly_all_data[col_list].sum(axis=1)*CONVERSION_FACTOR

monthly_all_data.head()

,date,coal,date1,natural gas,date2,nuclear,date3,hydroelectric,date4,wind,total
0,202209,24.98724,202209,10736.90153,202209,1611.663,202209,1732.27996,202209,887.64266,14993474.39
1,202208,19.65788,202208,11568.06973,202208,1679.400,202208,1756.68800,202208,1083.22678,16107042.39
2,202207,22.38795,202207,9504.45244,202207,1684.102,202207,1856.69645,202207,1443.68919,14511328.03
3,202206,19.56353,202206,7134.78680,202206,1633.288,202206,1816.49729,202206,1460.87295,12065008.57
4,202205,15.52536,202205,5208.03935,202205,1692.998,202205,1835.48466,202205,1814.01024,10566057.61


In [68]:
monthly_all_data = monthly_all_data.drop(['coal','date1','natural gas','date2','nuclear','date3','hydroelectric','date4','wind'], axis=1)
monthly_all_data = monthly_all_data.rename(columns={'total': 'demand'})

months_list = list(ca_monthly_emissions)
months_list.remove('year')

monthly_all_data['emissions'] = ''

for index, row in monthly_all_data.iterrows():
    curr_date = str(row['date'])
    curr_year = curr_date[0:4]
    curr_month = curr_date[4:6]
    if int(curr_year) in ca_monthly_emissions['year'].tolist():
        month_index = int(curr_month) - 1
        month_emissions = ca_monthly_emissions.loc[ca_monthly_emissions['year'] == int(curr_year), months_list[month_index]]
        monthly_all_data.loc[monthly_all_data['date'] == int(curr_date), 'emissions'] = float(month_emissions)

monthly_all_data = cleanup_null_lines(monthly_all_data, 'emissions')
monthly_all_data.head()


,date,demand,emissions
6,202203,10086091.07,3490823.36
7,202202,9625676.18,3513663.33
8,202201,10918131.70,4339494.42
9,202112,11489598.82,4959996.71
10,202111,10731961.28,4365100.13


In [69]:
# Include date index

monthly_all_data['date_idx'] = ''

dates_list = monthly_all_data['date'].tolist()
dates_list_chronological = dates_list.reverse()
dates_dict = {}

for i in range(len(dates_list)):
    dates_dict[dates_list[i]] = i

for index, row in monthly_all_data.iterrows():
    curr_date = str(row['date'])
    curr_year = curr_date[0:4]
    if int(curr_year) in ca_monthly_emissions['year'].tolist():
        monthly_all_data.loc[monthly_all_data['date'] == int(curr_date), 'date_idx'] = dates_dict[int(curr_date)]

monthly_all_data.head()

,date,demand,emissions,date_idx
6,202203,10086091.07,3490823.36,98
7,202202,9625676.18,3513663.33,97
8,202201,10918131.70,4339494.42,96
9,202112,11489598.82,4959996.71,95
10,202111,10731961.28,4365100.13,94


In [77]:
# Estimate monthly number of cars

MILLION_CONVERSION_FACTOR = 1000000

monthly_all_data['total'] = ''
for type in fuel_types:
    monthly_all_data[type] = ''

year_list = ca_yearly_vehicles['year'].tolist()
for index, row in ca_yearly_vehicles.iterrows():
    curr_year = row['year']
    prev_year = curr_year - 1
    if prev_year in year_list and curr_year in year_list:
        for type in fuel_types:
            curr_year_count = ca_yearly_vehicles.loc[ca_yearly_vehicles['year'] == curr_year, type]
            prev_year_count = ca_yearly_vehicles.loc[ca_yearly_vehicles['year'] == prev_year, type]
            increase = int(curr_year_count) - int(prev_year_count)
            increment = increase / 12
            monthly_total = 0
            for i in range(12):
                month = int(i) + 1
                month_string = str(month)
                if len(month_string) < 2:
                    month_string = '0' + month_string
                full_date = str(curr_year) + month_string
                monthly_value = int(prev_year_count) + int(increment) * int(month)
                monthly_all_data.loc[monthly_all_data['date'] == int(full_date), type] = int(float(monthly_value))
                monthly_total += int(float(monthly_value))

for index, row in monthly_all_data.iterrows():
    month_total = monthly_all_data.loc[monthly_all_data['date'] == row['date']][fuel_types].sum(axis=1)
    monthly_all_data.loc[monthly_all_data['date'] == row['date'], 'total'] = int(month_total)

monthly_all_data = cleanup_null_lines(monthly_all_data, 'electric')
monthly_all_data.to_csv('data/monthly_all_data.csv')
monthly_all_data.head()


,date,demand,emissions,date_idx,total,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane
9,202112,11489598.82,4959996.71,95,29942470,522436,305309,10117,590218,1208964,25998612,1298265,8456,93
10,202111,10731961.28,4365100.13,94,29836092,509680,301459,9868,599397,1203595,25917176,1286366,8391,160
11,202110,12062383.69,4577134.35,93,29729714,496924,297609,9619,608576,1198226,25835740,1274467,8326,227
12,202109,13537589.39,5070292.57,92,29623336,484168,293759,9370,617755,1192857,25754304,1262568,8261,294
13,202108,15754584.69,5607233.68,91,29516958,471412,289909,9121,626934,1187488,25672868,1250669,8196,361


**monthly_vehicle_percentages.csv**

In [83]:
monthly_vehicle_percentages = pd.read_csv('data/monthly_all_data.csv')
monthly_vehicle_percentages = monthly_vehicle_percentages.drop(['Unnamed: 0'], axis=1)

for index, row in monthly_vehicle_percentages.iterrows():
    for fuel in fuel_types:
        num_vehicle_type = row[fuel]
        total = row['total']
        percentage = num_vehicle_type / total * 100
        monthly_vehicle_percentages.loc[monthly_vehicle_percentages['date'] == row['date'], fuel] = percentage

monthly_vehicle_percentages.to_csv('data/monthly_all_data_percentages.csv')

**monthly_dem_em_weather.csv**

In [11]:
# Sum total electricity generation for each month, converted to megawatthours

CONVERSION_FACTOR = 1000

monthly_all_data = ca_monthly_supply
col_list = ['coal','natural gas','nuclear','hydroelectric','wind']
monthly_all_data['total'] = ''
monthly_all_data['total'] = monthly_all_data[col_list].sum(axis=1)*CONVERSION_FACTOR

monthly_all_data.head()

,date,coal,date1,natural gas,date2,nuclear,date3,hydroelectric,date4,wind,total
0,202209,24.98724,202209,10736.90153,202209,1611.663,202209,1732.27996,202209,887.64266,14993474.39
1,202208,19.65788,202208,11568.06973,202208,1679.400,202208,1756.68800,202208,1083.22678,16107042.39
2,202207,22.38795,202207,9504.45244,202207,1684.102,202207,1856.69645,202207,1443.68919,14511328.03
3,202206,19.56353,202206,7134.78680,202206,1633.288,202206,1816.49729,202206,1460.87295,12065008.57
4,202205,15.52536,202205,5208.03935,202205,1692.998,202205,1835.48466,202205,1814.01024,10566057.61


In [12]:
monthly_all_data = monthly_all_data.drop(['coal','date1','natural gas','date2','nuclear','date3','hydroelectric','date4','wind'], axis=1)
monthly_all_data = monthly_all_data.rename(columns={'total': 'demand'})

months_list = list(ca_monthly_emissions)
months_list.remove('year')

monthly_all_data['emissions'] = ''

for index, row in monthly_all_data.iterrows():
    curr_date = str(row['date'])
    curr_year = curr_date[0:4]
    curr_month = curr_date[4:6]
    if int(curr_year) in ca_monthly_emissions['year'].tolist():
        month_index = int(curr_month) - 1
        month_emissions = ca_monthly_emissions.loc[ca_monthly_emissions['year'] == int(curr_year), months_list[month_index]]
        monthly_all_data.loc[monthly_all_data['date'] == int(curr_date), 'emissions'] = float(month_emissions)

monthly_all_data = cleanup_null_lines(monthly_all_data, 'emissions')
monthly_all_data.head()


,date,demand,emissions
6,202203,10086091.07,3490823.36
7,202202,9625676.18,3513663.33
8,202201,10918131.70,4339494.42
9,202112,11489598.82,4959996.71
10,202111,10731961.28,4365100.13


In [30]:
DATE_START_IDX = 10
DATE_END_IDX = 18

minute_emissions_demand = pd.DataFrame()
minute_emissions_demand['emissions'] = ''

emissions_path_name = 'data/original/ca_daily_emissions'
cwd = os.path.abspath(emissions_path_name)
file_list = os.listdir(cwd)

demand_path_name = 'data/original/ca_daily_demand'
supply_path_name = 'data/original/ca_daily_supply'
renewables_path_name = 'data/original/ca_daily_renewables'

for file in file_list:
    date = file.replace('CAISO-co2-','').replace('.csv','')

    emission_full_path = emissions_path_name + '/' + file
    emissions_file = pd.read_csv(emission_full_path)
    emissions_file = emissions_file.drop(emissions_file.columns[0], axis=1)

    demand_file_name = file.replace('co2','netdemand')
    demand_full_path = demand_path_name + '/' + demand_file_name

    supply_file_name = file.replace('co2','supply')
    supply_full_path = supply_path_name + '/' + supply_file_name

    renewables_file_name = file.replace('co2','renewables')
    renewables_full_path = renewables_path_name + '/' + renewables_file_name

    if os.path.exists(demand_full_path) and os.path.exists(supply_full_path) and os.path.exists(renewables_full_path):
        emissions_transpose = emissions_file.transpose()

        demand_file = pd.read_csv(demand_full_path)
        demand_file = demand_file.loc[demand_file[demand_file.columns[0]] == 'Demand']
        demand_file = demand_file.drop(demand_file.columns[0], axis=1)
        demand_transpose = demand_file.transpose()

        supply_file = pd.read_csv(supply_full_path)
        supply_row_names = supply_file[supply_file.columns[0]]
        supply_transpose = supply_file.transpose()
        supply_transpose = supply_transpose.iloc[1: , :]
        for i in range(len(supply_row_names)):
            supply_transpose = supply_transpose.rename(columns={supply_transpose.columns[i]:supply_row_names[i]})
        supply_transpose = supply_transpose.drop('Renewables',axis=1)

        renewables_file = pd.read_csv(renewables_full_path)
        renewables_row_names = renewables_file[renewables_file.columns[0]]
        renewables_file = pd.read_csv(renewables_full_path)
        renewables_transpose = renewables_file.transpose()
        renewables_transpose = renewables_transpose.iloc[1: , :]
        for i in range(len(renewables_row_names)):
            renewables_transpose = renewables_transpose.rename(columns={renewables_transpose.columns[i]:renewables_row_names[i]})

        date_col = []
        for i in range(len(emissions_transpose)):
            date_col.append(date)

        merged = emissions_transpose.join(demand_transpose)        
        merged.rename(columns={merged.columns[0]:'emissions',merged.columns[1]:'demand'},inplace=True)
        merged.insert(loc=0, column="date", value=date)

        merged = merged.join(supply_transpose)
        merged = merged.join(renewables_transpose)

        if minute_emissions_demand.empty:
            minute_emissions_demand = merged
        else:
            minute_emissions_demand = pd.concat([minute_emissions_demand, merged])

minute_emissions_demand['total generated'] = minute_emissions_demand['Natural gas'] + minute_emissions_demand['Large hydro'] + minute_emissions_demand['Imports'] + minute_emissions_demand['Batteries'] + minute_emissions_demand['Nuclear'] + minute_emissions_demand['Coal'] + minute_emissions_demand['Other'] + minute_emissions_demand['Solar'] + minute_emissions_demand['Wind'] + minute_emissions_demand['Geothermal'] + minute_emissions_demand['Biomass'] + minute_emissions_demand['Biogas'] + minute_emissions_demand['Small hydro'] 

minute_emissions_demand.to_csv('data/cleaned/minute_emissions_demand_resource.csv')
minute_emissions_demand.head()

,date,emissions,demand,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Solar,Wind,Geothermal,Biomass,Biogas,Small hydro,total generated
00:00,20221112,8079,22291.0,14108,1075,4101,131,1130,2,0,4,535,949,241,213,111,22600
00:05,20221112,7997,22389.0,14092,951,3889,643,1130,2,0,4,533,949,242,213,111,22759
00:10,20221112,7915,22412.0,14040,912,3715,840,1130,2,0,4,517,949,242,213,113,22677
00:15,20221112,7874,22358.0,14157,933,3641,826,1130,3,0,4,507,949,242,213,115,22720
00:20,20221112,7876,22316.0,14057,874,3668,834,1130,3,0,4,506,949,240,213,118,22596


In [70]:
bay_area_emissions = pd.read_csv('data/original/bay_area_county_emissions.csv')
bay_area_emissions = bay_area_emissions[['County','Year','Pc_Ghg_Total_Metrictons']] 

bay_area_emissions.head()

,County,Year,Pc_Ghg_Total_Metrictons
0,Alameda,2014,6.133171
1,Contra Costa,2014,11.276073
2,Marin,2014,6.204910
3,Napa,2014,7.030953
4,San Francisco,2014,4.527429


In [137]:
monthly_demand = {}
ca_hourly_demand = pd.read_csv('data/original/ca_hourly_demand.csv')

for index, row in ca_hourly_demand.iterrows():
    # date format: 20150814T08Z
    curr_date = row['date']
    curr_year_month = curr_date[0:6]
    if curr_year_month not in monthly_demand:
        monthly_demand[curr_year_month] = row['demand']
    else:
        monthly_demand[curr_year_month] = monthly_demand[curr_year_month] + row['demand']

ca_monthly_demand = pd.DataFrame.from_dict(monthly_demand, orient='index')
ca_monthly_demand = ca_monthly_demand.rename(columns={0:'demand'})
ca_monthly_demand['date_idx'] = ''

size = ca_hourly_demand.shape[0]
for index, row in ca_hourly_demand.iterrows():
    ca_hourly_demand.loc[ca_hourly_demand.index == row.index, 'date_idx'] = size - index
# ca_monthly_demand.to_csv('data/cleaned/monthly_demand.csv')
ca_monthly_demand.head()

,demand,date_idx
202211,18098856,
202210,22583054,
202209,27240453,
202208,29002852,
202207,26935101,


In [182]:
NUM_MONTHS = 12
months = ca_monthly_emissions.columns

monthly_emissions_reformat = pd.DataFrame()

for index, row in ca_monthly_emissions.iterrows():
    year = row['year']
    for i in range(NUM_MONTHS):
        curr_month = i+1
        curr_column = months[curr_month]
        month_string = str(curr_month)
        if len(month_string) < 2:
            month_string = '0' + month_string
        full_date = str(int(year)) + month_string
        new_data = [full_date, row[months[curr_month]]]
        new_frame = pd.DataFrame([new_data], columns=['','emissions'])
        monthly_emissions_reformat = monthly_emissions_reformat.append(new_frame)

# monthly_emissions_reformat = monthly_emissions_reformat.drop(columns=[monthly_emissions_reformat['']])
monthly_emissions_reformat.to_csv('data/cleaned/monthly_emissions_reformat.csv',index=False)
monthly_emissions_reformat.head()

,,emissions
0,201401,5615497.820
0,201402,5022762.297
0,201403,5289480.524
0,201404,4812433.460
0,201405,5331965.630


In [13]:
# Estimate monthly number of cars

ca_yearly_vehicles = pd.read_csv('data/cleaned/yearly_vehicles_reformat.csv')
fuel_types = ['electric','plug-in hybrid electric','fuel cell','diesel','flex fuel','gasoline','gasoline hybrid','natural gas','propane']

monthly_vehicles = pd.DataFrame()
monthly_vehicles['date'] = ''
for type in fuel_types:
    monthly_vehicles[type] = ''

month_increment = {}
year_list = ca_yearly_vehicles['year'].tolist()
for index, row in ca_yearly_vehicles.iterrows():
    curr_year = row['year']
    prev_year = curr_year - 1
    if curr_year in year_list and prev_year in year_list:
        for i in reversed(range(12)):
            curr_month = str(i + 1)
            if len(curr_month) < 2:
                curr_month = '0' + curr_month
            full_date = str(curr_year) + curr_month
            new_row = [full_date]
            for type in fuel_types:
                starting_num = row[type]
                type_year = type + str(curr_year)
                if type_year not in month_increment:
                    prev_num = ca_yearly_vehicles.loc[ca_yearly_vehicles['year'] == prev_year, type]
                    increment_size = (starting_num - prev_num) / 12
                    month_increment[type_year] = increment_size
                else:
                    increment_size = month_increment[type_year]
                increment = 12 - i - 1
                month_num = int(starting_num - increment_size * increment)
                new_row.append(month_num)
            new_frame = pd.DataFrame([new_row],columns=monthly_vehicles.columns)  
            monthly_vehicles = monthly_vehicles.append(new_frame)

monthly_vehicles = monthly_vehicles.rename(columns={'natural gas':'natural gas fuel'})
monthly_vehicles.to_csv('data/cleaned/monthly_vehicles.csv')
monthly_vehicles.head()


,date,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas fuel,propane
0,202112,522445,305315,10127,590216,1208970,25998618,1298275,8461,90
0,202111,509688,301464,9877,599395,1203600,25917181,1286375,8395,157
0,202110,496931,297614,9627,608574,1198231,25835745,1274475,8330,224
0,202109,484174,293763,9377,617753,1192861,25754308,1262575,8264,291
0,202108,471418,289913,9127,626932,1187492,25672872,1250675,8199,359


In [75]:
tmax = pd.read_csv('data/cleaned/monthly_tmax.csv')
tmax = tmax.drop(columns=['Unnamed: 0'])
tmax.head()

,year,county code,January,February,March,April,May,June,July,August,September,October,November,December
0,2015,1,61.0,65.4,70.8,69.6,68.8,81.9,82.8,83.7,84.0,78.7,60.8,54.8
1,2016,1,56.2,65.5,64.1,69.6,73.6,82.8,83.8,81.9,81.6,71.6,64.1,55.4
2,2017,1,53.6,58.2,65.4,66.8,74.9,81.2,86.8,84.0,83.0,76.6,63.6,59.9
3,2018,1,58.7,62.5,61.3,67.3,70.6,80.8,85.9,82.2,80.4,75.5,66.2,56.6
4,2019,1,58.4,53.3,60.6,69.8,68.4,81.8,83.1,85.6,80.6,75.1,67.0,56.4


In [90]:
tmax_reformat = pd.DataFrame()

months = ['January','February','March','April','May','June','July','August','September','October','November','December']

county_codes_list = tmax['county code'].unique()

for code in county_codes_list:
    curr_code_rows = tmax.loc[tmax['county code'] == code]
    county_data = pd.DataFrame()
    for index, row in curr_code_rows.iterrows():
        year = int(row['year'])
        year_data = pd.DataFrame()
        for i in range(12):
            month_idx = i+2
            month = str(i+1)
            if len(month) < 2:
                month = '0' + month
            year_month = str(year) + month
            county_name = 'county ' + str(code) + ' tmax'
            new_data = pd.DataFrame({'date': [year_month], county_name: [row[month_idx]]})
            if year_data.empty:
                year_data = new_data
            else:
                year_data = pd.concat([year_data, new_data])
        if county_data.empty:
            county_data = year_data
        else:
            county_data = pd.concat([county_data, year_data])
    if tmax_reformat.empty:
        tmax_reformat = county_data
    else:
        tmax_reformat = tmax_reformat.merge(county_data)

tmax_reformat = tmax_reformat[~tmax_reformat.eq(-99.9).any(1)] # remove rows with unknown values, represented by -99.9

tmax_reformat['mean tmax'] = tmax_reformat.iloc[:, 1:].mean(axis=1)
tmax_reformat.to_csv('data/cleaned/monthly_tmax_reformat.csv')
print(tmax_reformat.shape)
tmax_reformat.head()


(95, 60)


,date,county 1 tmax,county 3 tmax,county 5 tmax,county 7 tmax,county 9 tmax,county 11 tmax,county 13 tmax,county 15 tmax,county 17 tmax,...,county 99 tmax,county 101 tmax,county 103 tmax,county 105 tmax,county 107 tmax,county 109 tmax,county 111 tmax,county 113 tmax,county 115 tmax,mean tmax
0,201501,61.0,48.5,59.9,61.0,60.2,61.9,60.1,54.7,57.2,...,58.2,60.6,61.0,54.5,53.1,54.4,64.1,61.2,61.4,59.558621
1,201502,65.4,50.9,63.7,65.3,64.8,67.3,66.1,55.9,59.4,...,66.3,67.8,63.5,57.6,60.8,57.4,66.4,67.3,66.3,64.100000
2,201503,70.8,53.5,69.3,71.3,70.0,74.6,72.2,60.7,63.8,...,74.3,76.1,69.1,64.1,65.5,60.7,71.4,74.5,72.5,69.410345
3,201504,69.6,52.2,68.4,72.2,69.3,76.2,71.6,60.9,63.1,...,74.8,78.1,70.8,64.9,65.1,59.6,68.6,76.8,73.3,69.055172
4,201505,68.8,55.9,72.7,76.8,73.2,80.0,71.1,67.3,67.5,...,77.6,81.5,75.4,71.7,67.9,62.3,68.7,79.4,77.7,71.446552


In [79]:
precip = pd.read_csv('data/cleaned/monthly_precip.csv')
precip = precip.drop(columns=['Unnamed: 0'])
precip.head()

,year,county code,January,February,March,April,May,June,July,August,September,October,November,December
0,2015,1,0.01,2.25,0.26,1.15,0.37,0.27,0.01,0.02,0.04,0.08,2.41,3.52
1,2016,1,5.65,0.84,5.25,1.94,0.24,0.00,0.00,0.00,0.00,3.65,1.48,3.50
2,2017,1,9.38,7.41,2.65,2.50,0.03,0.04,0.00,0.01,0.06,0.21,2.58,0.10
3,2018,1,4.28,0.57,4.39,2.44,0.02,0.00,0.00,0.00,0.00,0.14,2.71,2.02
4,2019,1,3.73,6.83,4.11,0.47,2.25,0.00,0.00,0.00,0.19,0.00,1.09,3.62


In [45]:
precip_reformat = pd.DataFrame()

months = ['January','February','March','April','May','June','July','August','September','October','November','December']

county_codes_list = precip['county code'].unique()

for code in county_codes_list:
    curr_code_rows = precip.loc[precip['county code'] == code]
    county_data = pd.DataFrame()
    for index, row in curr_code_rows.iterrows():
        year = int(row['year'])
        year_data = pd.DataFrame()
        for i in range(12):
            month_idx = i+2
            month = str(i+1)
            if len(month) < 2:
                month = '0' + month
            year_month = str(year) + month
            county_name = 'county ' + str(code) + ' precip'
            new_data = pd.DataFrame({'date': [year_month], county_name: [row[month_idx]]})
            if year_data.empty:
                year_data = new_data
            else:
                year_data = pd.concat([year_data, new_data])
        if county_data.empty:
            county_data = year_data
        else:
            county_data = pd.concat([county_data, year_data])
    if precip_reformat.empty:
        precip_reformat = county_data
    else:
        precip_reformat = precip_reformat.merge(county_data)


precip_reformat = precip_reformat[~precip_reformat.eq(-99.9).any(1)] # remove rows with unknown values, represented by -99.9
precip_reformat['mean precip'] = precip_reformat.iloc[:, 1:].mean(axis=1)
precip_reformat.to_csv('data/cleaned/monthly_precip_reformat.csv')
print(precip_reformat.shape)
precip_reformat.head()


(96, 60)


,date,county 1 precip,county 3 precip,county 5 precip,county 7 precip,county 9 precip,county 11 precip,county 13 precip,county 15 precip,county 17 precip,...,county 99 precip,county 101 precip,county 103 precip,county 105 precip,county 107 precip,county 109 precip,county 111 precip,county 113 precip,county 115 precip,mean precip
0,201501,0.01,0.22,0.14,0.21,0.13,0.18,0.01,3.49,0.12,...,0.02,0.10,0.25,1.79,0.47,0.14,1.97,0.06,0.08,0.440345
1,201502,2.25,5.53,5.36,4.58,5.26,3.66,2.34,11.17,6.96,...,1.96,2.49,4.29,11.74,2.29,5.12,0.88,3.20,4.42,3.597241
2,201503,0.26,0.62,0.83,0.58,0.64,0.54,0.26,5.65,0.87,...,0.23,0.17,1.04,2.53,0.43,0.68,0.69,0.17,0.48,0.722759
3,201504,1.15,2.52,3.09,2.24,2.64,1.78,1.09,4.36,3.41,...,0.77,1.52,1.76,2.03,0.87,3.29,0.32,1.31,2.75,1.556897
4,201505,0.37,2.99,1.39,0.29,1.18,0.08,0.20,0.68,1.96,...,0.23,0.03,0.61,1.23,1.76,2.70,0.61,0.10,0.17,0.862759


In [8]:
monthly_all_data = pd.read_csv('data/cleaned/monthly_all_data.csv')
monthly_all_data = monthly_all_data.drop(columns=['Unnamed: 0','emissions','demand'])

monthly_all_data.to_csv('data/cleaned/monthly_all_data.csv')
monthly_all_data.head()

,date,date_idx,total,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane
0,202112,95,29942470,522436,305309,10117,590218,1208964,25998612,1298265,8456,93
1,202111,94,29836092,509680,301459,9868,599397,1203595,25917176,1286366,8391,160
2,202110,93,29729714,496924,297609,9619,608576,1198226,25835740,1274467,8326,227
3,202109,92,29623336,484168,293759,9370,617755,1192857,25754304,1262568,8261,294
4,202108,91,29516958,471412,289909,9121,626934,1187488,25672868,1250669,8196,361


In [11]:
monthly_vehicles_percentages = pd.read_csv('data/cleaned/monthly_vehicles_percentages.csv')
monthly_vehicles_percentages = monthly_vehicles_percentages.drop(columns=['Unnamed: 0','emissions','demand'])

monthly_vehicles_percentages.to_csv('data/cleaned/monthly_vehicles_percentages.csv')
monthly_vehicles_percentages.head()

,date,date_idx,total,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane
0,202112,95,29942470,1.744799,1.019652,0.033788,1.971173,4.037623,86.828548,4.335865,0.028241,0.000311
1,202111,94,29836092,1.708267,1.010384,0.033074,2.008966,4.034024,86.865183,4.311443,0.028124,0.000536
2,202110,93,29729714,1.671473,1.001049,0.032355,2.047029,4.030399,86.902081,4.286846,0.028006,0.000764
3,202109,92,29623336,1.634414,0.991647,0.031630,2.085366,4.026748,86.939243,4.262072,0.027887,0.000992
4,202108,91,29516958,1.597089,0.982178,0.030901,2.123979,4.023070,86.976673,4.237120,0.027767,0.001223


In [27]:
monthly_vehicles = pd.read_csv('data/cleaned/monthly_vehicles.csv')
monthly_vehicles = monthly_vehicles.drop(columns=['Unnamed: 0'])
monthly_vehicles['total'] = ''
monthly_vehicles['date_idx'] = ''

monthly_vehicles['total'] = monthly_vehicles['electric'] + monthly_vehicles['plug-in hybrid electric'] + monthly_vehicles['fuel cell']+ monthly_vehicles['diesel']+ monthly_vehicles['flex fuel']+ monthly_vehicles['gasoline']+ monthly_vehicles['gasoline hybrid']+ monthly_vehicles['natural gas fuel']+ monthly_vehicles['propane']

size = monthly_vehicles.shape[0]
for index, row in monthly_vehicles.iterrows():
    date_idx = size - index
    monthly_vehicles.loc[monthly_vehicles['date'] == row['date'] , 'date_idx'] = date_idx

monthly_vehicles.to_csv('data/cleaned/monthly_vehicles.csv')
monthly_vehicles.head()

,date,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas fuel,propane,total,date_idx
0,202112,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,29942517,132
1,202111,509688,301464,9877,599395,1203600,25917181,1286375,8395,157,29836132,131
2,202110,496931,297614,9627,608574,1198231,25835745,1274475,8330,224,29729751,130
3,202109,484174,293763,9377,617753,1192861,25754308,1262575,8264,291,29623366,129
4,202108,471418,289913,9127,626932,1187492,25672872,1250675,8199,359,29516987,128


In [40]:
tmax = pd.read_csv('data/cleaned/monthly_tmax_reformat.csv')
tmax = tmax.drop(columns=['Unnamed: 0'])
tmax['date_idx'] = ''

for index, row in tmax.iterrows():
    tmax.loc[tmax['date'] == row['date'] , 'date_idx'] = index + 1

tmax.to_csv('data/cleaned/monthly_tmax_reformat.csv')
tmax.head()

,date,county 1 tmax,county 3 tmax,county 5 tmax,county 7 tmax,county 9 tmax,county 11 tmax,county 13 tmax,county 15 tmax,county 17 tmax,...,county 101 tmax,county 103 tmax,county 105 tmax,county 107 tmax,county 109 tmax,county 111 tmax,county 113 tmax,county 115 tmax,mean tmax,date_idx
0,201501,61.0,48.5,59.9,61.0,60.2,61.9,60.1,54.7,57.2,...,60.6,61.0,54.5,53.1,54.4,64.1,61.2,61.4,59.558621,1
1,201502,65.4,50.9,63.7,65.3,64.8,67.3,66.1,55.9,59.4,...,67.8,63.5,57.6,60.8,57.4,66.4,67.3,66.3,64.100000,2
2,201503,70.8,53.5,69.3,71.3,70.0,74.6,72.2,60.7,63.8,...,76.1,69.1,64.1,65.5,60.7,71.4,74.5,72.5,69.410345,3
3,201504,69.6,52.2,68.4,72.2,69.3,76.2,71.6,60.9,63.1,...,78.1,70.8,64.9,65.1,59.6,68.6,76.8,73.3,69.055172,4
4,201505,68.8,55.9,72.7,76.8,73.2,80.0,71.1,67.3,67.5,...,81.5,75.4,71.7,67.9,62.3,68.7,79.4,77.7,71.446552,5


In [46]:
precip = pd.read_csv('data/cleaned/monthly_precip_reformat.csv')
precip = precip.drop(columns=['Unnamed: 0'])
precip['date_idx'] = ''

for index, row in precip.iterrows():
    precip.loc[precip['date'] == row['date'] , 'date_idx'] = index + 1

precip.to_csv('data/cleaned/monthly_precip_reformat.csv')
precip.head()

,date,county 1 precip,county 3 precip,county 5 precip,county 7 precip,county 9 precip,county 11 precip,county 13 precip,county 15 precip,county 17 precip,...,county 101 precip,county 103 precip,county 105 precip,county 107 precip,county 109 precip,county 111 precip,county 113 precip,county 115 precip,mean precip,date_idx
0,201501,0.01,0.22,0.14,0.21,0.13,0.18,0.01,3.49,0.12,...,0.10,0.25,1.79,0.47,0.14,1.97,0.06,0.08,0.440345,1
1,201502,2.25,5.53,5.36,4.58,5.26,3.66,2.34,11.17,6.96,...,2.49,4.29,11.74,2.29,5.12,0.88,3.20,4.42,3.597241,2
2,201503,0.26,0.62,0.83,0.58,0.64,0.54,0.26,5.65,0.87,...,0.17,1.04,2.53,0.43,0.68,0.69,0.17,0.48,0.722759,3
3,201504,1.15,2.52,3.09,2.24,2.64,1.78,1.09,4.36,3.41,...,1.52,1.76,2.03,0.87,3.29,0.32,1.31,2.75,1.556897,4
4,201505,0.37,2.99,1.39,0.29,1.18,0.08,0.20,0.68,1.96,...,0.03,0.61,1.23,1.76,2.70,0.61,0.10,0.17,0.862759,5


In [110]:
new_monthly_vehicles = pd.read_csv('data/cleaned/monthly_vehicles.csv')
new_monthly_vehicles = new_monthly_vehicles.drop(columns=['Unnamed: 0'])
new_monthly_vehicles.head()

,date,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas fuel,propane,total,date_idx
0,202112,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,29942517,132
1,202111,509688,301464,9877,599395,1203600,25917181,1286375,8395,157,29836132,131
2,202110,496931,297614,9627,608574,1198231,25835745,1274475,8330,224,29729751,130
3,202109,484174,293763,9377,617753,1192861,25754308,1262575,8264,291,29623366,129
4,202108,471418,289913,9127,626932,1187492,25672872,1250675,8199,359,29516987,128


In [111]:
yearly_vehicles = pd.read_csv('data/cleaned/yearly_vehicles_reformat.csv')
us_monthly_sales = pd.read_csv('data/original/us_monthly_car_sales.csv')
us_monthly_sales = us_monthly_sales.drop(columns=['Unnamed: 5'])
print(us_monthly_sales.shape)
us_monthly_sales.head()

(143, 5)


,date,electric,plug-in hybrid electric,gasoline hybrid,total LDV
0,201101,103,321,19540,819938
1,201102,83,281,23306,993535
2,201103,298,608,34533,1246668
3,201104,573,493,25602,1157928
4,201105,1150,481,17419,1061841


In [118]:
listed_monthly_vehicle_sales = ['electric','plug-in hybrid electric','gasoline hybrid']

year_totals = {}
for index, row in us_monthly_sales.iterrows():
    year = str(row['date'])[0:4]
    if year not in year_totals:
        year_totals[year] = {}
    for col in listed_monthly_vehicle_sales:
        if col not in year_totals[year]:
            year_totals[year][col] = row[col]
        else:
            year_totals[year][col] += row[col]

for index, row in new_monthly_vehicles.iterrows():
    date = row['date']
    month = str(int(row['date']))[4:]
    year = str(row['date'])[0:4]

    next_month = int(month) + 1
    next_month_str = str(next_month)
    if len(next_month_str) < 2:
        next_month_str = '0' + next_month_str
    next_month_full = str(year) + next_month_str

    for col in listed_monthly_vehicle_sales:
        prev_year = int(year)-1
        year_reg = int(yearly_vehicles.loc[yearly_vehicles['year']==int(year),col])
        prev_year_reg = int(yearly_vehicles.loc[yearly_vehicles['year']==prev_year,col])
        year_diff = year_reg - prev_year_reg
        if month == '12':
            new_count = year_reg
        else:
            year_sales = int(year_totals[year][col])
            month_sales = int(us_monthly_sales.loc[us_monthly_sales['date'] == int(next_month_full), col])
            proportion = month_sales/year_sales
            increment = int(year_diff * proportion)
            next_month_count = int(new_monthly_vehicles.loc[new_monthly_vehicles['date'] == int(next_month_full), col])
            new_count = next_month_count - increment
        # print(new_count)
        
        new_monthly_vehicles.loc[new_monthly_vehicles['date'] == date, col] = new_count

new_monthly_vehicles.to_csv('data/cleaned/new_monthly_vehicles.csv')
new_monthly_vehicles.head()



,date,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas fuel,propane,total,date_idx
0,202112,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,29942517,132
1,202111,505951,300973,9877,599395,1203600,25917181,1285769,8395,157,29836132,131
2,202110,490400,297256,9627,608574,1198231,25835745,1275167,8330,224,29729751,130
3,202109,476248,292462,9377,617753,1192861,25754308,1263823,8264,291,29623366,129
4,202108,462251,289169,9127,626932,1187492,25672872,1253083,8199,359,29516987,128


In [130]:
new_demand = pd.read_csv('data/cleaned/monthly_demand.csv')
new_demand['date_idx'] = ''

size = new_demand.shape[0]
for index, row in new_demand.iterrows():
    new_demand.loc[new_demand['Unnamed: 0'] == row['Unnamed: 0'], 'date_idx'] = size - index

new_demand.to_csv('data/cleaned/new_monthly_demand.csv')
new_demand.head()

,Unnamed: 0,demand,date_idx
0,202211,18098856,89
1,202210,22583054,88
2,202209,27240453,87
3,202208,29002852,86
4,202207,26935101,85
